## 🔍 Step 4: Basic RAG Implementation

Now we'll set up the basic RAG pipeline with OpenAI LLM and Pinecone retriever.

**Progress**: Initializing LLM and creating basic RAG function...

In [ ]:
# Basic RAG Implementation
from langchain_openai import ChatOpenAI
from langchain_pinecone import PineconeVectorStore
from langchain_community.embeddings import SentenceTransformerEmbeddings
import numpy as np
import os
from dotenv import load_dotenv
from pinecone import Pinecone

load_dotenv()
# Get OpenAI API key
openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key:
    raise ValueError("OPENAI_API_KEY not found in environment variables or .env file")

print("✅ OpenAI API key loaded successfully")

# Initialize OpenAI LLM
llm = ChatOpenAI(
    model="gpt-4o-mini",  # Using GPT-4o-mini for cost efficiency
    openai_api_key=openai_api_key,
    temperature=0.1,
    streaming=False
)

print("✅ OpenAI LLM initialized successfully")
print(f"🤖 Model: gpt-4o-mini")
print(f"🌡️ Temperature: 0.1")
print(f"📡 Streaming: False")

# Test the LLM with a simple query
try:
    test_response = llm.invoke("Hello! Please respond with 'LLM is working correctly.'")
    print(f"\n🧪 Test Response: {test_response.content}")
    print("✅ LLM is ready to use!")

except Exception as e:
    print(f"❌ Error testing LLM: {str(e)}")
    print("Please check your API key and internet connection.")

# Initialize Pinecone
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
index_name = os.getenv("PINECONE_INDEX")
index = pc.Index(index_name)

# Initialize embedding model
embedding_model = SentenceTransformerEmbeddings(
    model_name='intfloat/multilingual-e5-large'
)

# Create VectorStore
vectorstore = PineconeVectorStore(
    index=index,
    embedding=embedding_model
)

# Create retriever with similarity search and k=5
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}
)

print("✅ Setup complete! Pinecone retriever ready.")
print(f"📊 Index name: {index_name}")
print(f"🔍 Search type: similarity, k=5")
print(f"🤖 Embedding model: intfloat/multilingual-e5-large")

✅ OpenAI API key loaded successfully
✅ OpenAI LLM initialized successfully
🤖 Model: gpt-4o-mini
🌡️ Temperature: 0.1
📡 Streaming: False

🧪 Test Response: LLM is working correctly.
✅ LLM is ready to use!


NameError: name 'Pinecone' is not defined

In [ ]:
def get_rag_answer(query: str, retriever, llm) -> str:
    """
    Retrieve relevant chunks and generate answer using OpenAI LLM

    Args:
        query: User's question
        retriever: Pinecone retriever or custom retrieval function
        llm: OpenAI LLM instance

    Returns:
        Generated answer based on retrieved context
    """

    # Method 1: Try using LangChain retriever first
    try:
        query_embedding = embedding_model.embed_query(query)
        response = index.query(
            vector=query_embedding,
            top_k=5,
            include_metadata=True,
            include_values=False
        )
        chunks = []
        metadata_list = []
        for match in response.matches:
            chunk_text = match.metadata.get('chunk_text', '')
            if chunk_text:
                chunks.append(chunk_text)
                metadata_list.append(match.metadata)
        print(f"✅ Retrieved {len(chunks)} chunks using direct Pinecone query")
        if not chunks:
          return "No relevant information found in the database."
    except Exception as e:
         print("Exception",e)
    # Print retrieved metadata for transparency
    print("\n📋 RETRIEVED CHUNKS METADATA:")
    print("-" * 50)
    for i, metadata in enumerate(metadata_list, 1):
        company = metadata.get('company', 'Unknown').replace(' (1)', '')
        year = metadata.get('year', 'Unknown')
        chunk_id = metadata.get('chunk_id', 'Unknown')
        source = metadata.get('section', 'Unknown')
        chunk_text=metadata.get('chunk_text', 'Unknown')
        print(f"Chunk {i}: {company.title()} ({year}) - ID: {chunk_id} - {source}")
        #print(f"Chunk text {i}: {chunk_text}")

    # Combine chunks into context
    context = "\n\n".join([f"Document {i+1}:\n{chunk}" for i, chunk in enumerate(chunks)])

    # Create prompt for OpenAI
    prompt = f"""Based on the following documents, please answer the user's question accurately and comprehensively.

QUESTION: {query}

CONTEXT DOCUMENTS:
{context}

INSTRUCTIONS:
- Use only the information provided in the context documents
- If the information is not sufficient to answer the question, state this clearly
- Provide specific details and numbers when available
- Structure your answer clearly and concisely
- If data spans multiple years or sources, organize it logically

ANSWER:"""

    # Send to OpenAI LLM
    try:
        response = llm.invoke(prompt)
        answer = response.content.strip()

        print(f"\n🤖 OpenAI LLM Response Generated ({len(answer)} characters)")
        return answer

    except Exception as e:
        return f"Error generating answer with OpenAI: {str(e)}"

# Test the basic RAG function
print("🧪 Testing basic RAG function...")
test_query = "Summarize key points from Apple 10k."
final_answer = get_rag_answer(test_query, retriever, llm)
print("\n" + "="*80)
print("🎯 BASIC RAG ANSWER:")
print("="*80)
print(final_answer)
print("="*80)

print(f"\n✅ Step 4 Complete: Basic RAG implementation finished!")

🧪 Testing basic RAG function...


NameError: name 'retriever' is not defined